# Julemandens workshop

Annotér nogle billeder og træn en model, så vi automatisk kan finde ud af, hvilke af de mange millioner gaver, vi stadig kan bruge efter indbruddet.

## Opstart

Det her dokument er en "Notebook".  
Notebook'en består af celler med kode i. 
* Du kan aktivere en celle ved at trykke inde i cellen
* DU kan køre en celle ved at aktivere den og trykke "shift + enter"

Vi skal bruge de nedenstående celler.  
Aktiver den første celle herunder og tryk "shift + enter".  
Cellen går automatisk videre til den næste celle.  
Tryk "shift + enter" igen for også at køre den.

In [ ]:
%load_ext autoreload
%autoreload 2
%pip install -q requirements.txt

In [ ]:
import albumentations as A
import utils
import annoterer
import træning

## Annoter Billeder

Der er 500 billeder, du kan annotere, men start med 200.  
Vi bruger 100 billeder til vores valideringssæt, så med 200 billeder totalt er der 100 billeder til træning.  
Du kan altid komme tilbage og annotere flere ved at sætte "antal_billeder" op og køre cellen igen.
Hvis du f.eks. annoterer 300 billeder, er der 200 til træning. 

Start med at køre cellen herunder og annotér 200 billeder:

In [ ]:
annoterer.annoter(antal_billeder=200)

## Billedeaugmenteringer

Med billedeaugmentering laver vi ændringer i billederne i vores træningsdata, uden at billedets label ændrer sig.  
På den måde får vi mere data at træne på uden at skulle annotere flere billeder.

Til at starte med kan du prøve uden annoteringer, men prøv at kom tilbage hertil, hvis du har problemer med overfitting.

Du kan køre den nedenstående celle for at se, hvordan billederne ser ud med forskellige augmenteringer.  
Prøv f.eks. at fjerne "#" i en af linjerne i cellen herunder og kør cellen igen:

In [ ]:
augmenteringer = [
    #A.RandomResizedCrop(224, 224, scale=(0.5, 1)),
    #A.ColorJitter(),   
    #A.RandomRotate90(),
    #A.Flip(),
]
utils.vis_augmenteringseksempler(augmenteringer)

## Træn en model

Kør den nedenstående celle for at træne din første model.

In [ ]:
model, version = træning.træn_model(
    epoker=20, 
    start_fra_prætrænede_vægte=False,
    augmenteringer=[],
    antal_træningsbilleder=100,
    model_størrelse='lille',
)
utils.plot_log(version)

Din model ser ud til at virke godt på dit træningsdata, men ikke særlig godt på dit valideringdata - et klart tegn på overfitting!  
Hvis vi skal bruge modellen til at hjælpe med de mange millioner af gaver, er det vigtigt, at modellen også virker på data, modellen ikke er trænet på!

Her er nogle ting, du kan eksperimentere med for at opnå en bedre model:

* Prøv at starte fra en model, der er trænet på et andet på problem med mange flere billeder (sæt "start_fra_prætrænede_vægte" til "True")
* Prøv at eksperimentér med data augmenteringer. (Du kan se eksempler på augmenteringer i en celle lidt højere oppe i dokumentet).
* Prøv at annotere nogle flere billeder og sæt "antal_træningsbilleder" op.
* Prøv forskellige modelstørrelser: "lille", "mellem" eller "stor".

Du kan enten ændre direkte i cellen ovenover eller kopiere cellen til nye celler herunder, og køre dem.